In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 42000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:43000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6587385146511437828-2
Page load happened
42000 https://kw.com/agent/UPA-6587385146511437828-2
***********************************
https://kw.com/agent/UPA-6778052651776258048-8
Page load happened
42001 https://kw.com/agent/UPA-6778052651776258048-8
***********************************
https://kw.com/agent/UPA-6587385265266774021-5
Page load happened
42002 https://kw.com/agent/UPA-6587385265266774021-5
***********************************
https://kw.com/agent/UPA-6587385381048528900-6
Page load happened
42003 https://kw.com/agent/UPA-6587385381048528900-6
***********************************
https://kw.com/agent/UPA-6587385398877945863-0
Page load happened
42004 https://kw.com/agent/UPA-6587385398877945863-0
***********************************
https://kw.com/agent/UPA-6820888919080124416-6
Page load happened
42005 https://kw.com/agent/UPA-6820888919080124416-6
***********************************
https://kw.com/agent/UPA-6800791344857464832-8
Timeout happened no pag

Page load happened
42053 https://kw.com/agent/UPA-6587385390463881222-7
***********************************
https://kw.com/agent/UPA-6592969400781795331-9
Timeout happened no page load
42054 https://kw.com/agent/UPA-6592969400781795331-9
***********************************
https://kw.com/agent/UPA-6587385162152792066-2
Page load happened
42055 https://kw.com/agent/UPA-6587385162152792066-2
***********************************
https://kw.com/agent/UPA-6638119406301872131-6
Page load happened
42056 https://kw.com/agent/UPA-6638119406301872131-6
***********************************
https://kw.com/agent/UPA-6592262121526816769-6
Page load happened
42057 https://kw.com/agent/UPA-6592262121526816769-6
***********************************
https://kw.com/agent/UPA-6587385162824773639-3
Page load happened
42058 https://kw.com/agent/UPA-6587385162824773639-3
***********************************
https://kw.com/agent/UPA-6587385261699608578-3
Page load happened
42059 https://kw.com/agent/UPA-658738526

Page load happened
42106 https://kw.com/agent/UPA-6587385204301266948-2
***********************************
https://kw.com/agent/UPA-6587385070645248002-5
Page load happened
42107 https://kw.com/agent/UPA-6587385070645248002-5
***********************************
https://kw.com/agent/UPA-6587385385717997573-5
Page load happened
42108 https://kw.com/agent/UPA-6587385385717997573-5
***********************************
https://kw.com/agent/UPA-6587385255808405505-6
Page load happened
42109 https://kw.com/agent/UPA-6587385255808405505-6
***********************************
https://kw.com/agent/UPA-6587385103945515014-0
Page load happened
42110 https://kw.com/agent/UPA-6587385103945515014-0
***********************************
https://kw.com/agent/UPA-6587385367969091585-5
Page load happened
42111 https://kw.com/agent/UPA-6587385367969091585-5
***********************************
https://kw.com/agent/UPA-6724779974008754176-7
Timeout happened no page load
42112 https://kw.com/agent/UPA-672477997

Page load happened
42159 https://kw.com/agent/UPA-6587385275006590981-4
***********************************
https://kw.com/agent/UPA-6587385167849177088-0
Page load happened
42160 https://kw.com/agent/UPA-6587385167849177088-0
***********************************
https://kw.com/agent/UPA-6677291825132617728-5
Page load happened
42161 https://kw.com/agent/UPA-6677291825132617728-5
***********************************
https://kw.com/agent/UPA-6587385434595082248-0
Page load happened
42162 https://kw.com/agent/UPA-6587385434595082248-0
***********************************
https://kw.com/agent/UPA-6587385286488899592-1
Page load happened
42163 https://kw.com/agent/UPA-6587385286488899592-1
***********************************
https://kw.com/agent/UPA-6587385365690953734-0
Page load happened
42164 https://kw.com/agent/UPA-6587385365690953734-0
***********************************
https://kw.com/agent/UPA-6712126730285559808-7
Page load happened
42165 https://kw.com/agent/UPA-6712126730285559808-

Page load happened
42212 https://kw.com/agent/UPA-6749725088849444864-5
***********************************
https://kw.com/agent/UPA-6788576180951867392-7
Page load happened
42213 https://kw.com/agent/UPA-6788576180951867392-7
***********************************
https://kw.com/agent/UPA-6724160607901425664-1
Page load happened
42214 https://kw.com/agent/UPA-6724160607901425664-1
***********************************
https://kw.com/agent/UPA-6587385417843445761-7
Page load happened
42215 https://kw.com/agent/UPA-6587385417843445761-7
***********************************
https://kw.com/agent/UPA-6587385375031771142-6
Page load happened
42216 https://kw.com/agent/UPA-6587385375031771142-6
***********************************
https://kw.com/agent/UPA-6663368350402736128-3
Page load happened
42217 https://kw.com/agent/UPA-6663368350402736128-3
***********************************
https://kw.com/agent/UPA-6587385371687395332-3
Page load happened
42218 https://kw.com/agent/UPA-6587385371687395332-

...

Page load happened
42742 https://kw.com/agent/UPA-6611573031978274817-5
***********************************
https://kw.com/agent/UPA-6752985835727945728-3
Page load happened
42743 https://kw.com/agent/UPA-6752985835727945728-3
***********************************
https://kw.com/agent/UPA-6587385404746555395-0
Page load happened
42744 https://kw.com/agent/UPA-6587385404746555395-0
***********************************
https://kw.com/agent/UPA-6631251642328334341-2
Page load happened
42745 https://kw.com/agent/UPA-6631251642328334341-2
***********************************
https://kw.com/agent/UPA-6605415560286928898-1
Page load happened
42746 https://kw.com/agent/UPA-6605415560286928898-1
***********************************
https://kw.com/agent/UPA-6837120919755280384-2
Page load happened
42747 https://kw.com/agent/UPA-6837120919755280384-2
***********************************
https://kw.com/agent/UPA-6641069270321188866-0
Page load happened
42748 https://kw.com/agent/UPA-6641069270321188866-

Page load happened
42795 https://kw.com/agent/UPA-6587385096501153793-4
***********************************
https://kw.com/agent/UPA-6592982330692247557-2
Page load happened
42796 https://kw.com/agent/UPA-6592982330692247557-2
***********************************
https://kw.com/agent/UPA-6587385051689250819-5
Page load happened
42797 https://kw.com/agent/UPA-6587385051689250819-5
***********************************
https://kw.com/agent/UPA-6651515711535992835-3
Page load happened
42798 https://kw.com/agent/UPA-6651515711535992835-3
***********************************
https://kw.com/agent/UPA-6775907610206953472-3
Page load happened
42799 https://kw.com/agent/UPA-6775907610206953472-3
***********************************
https://kw.com/agent/UPA-6587385052902391809-5
need to close driver
new driver initialize****************
Page load happened
42800 https://kw.com/agent/UPA-6587385052902391809-5
***********************************
https://kw.com/agent/UPA-6720431134800506880-0
Page load h

Page load happened
42848 https://kw.com/agent/UPA-6649341569860218883-3
***********************************
https://kw.com/agent/UPA-6709515348971114496-8
Page load happened
42849 https://kw.com/agent/UPA-6709515348971114496-8
***********************************
https://kw.com/agent/UPA-6587385384137920516-0
Page load happened
42850 https://kw.com/agent/UPA-6587385384137920516-0
***********************************
https://kw.com/agent/UPA-6587385291488333824-0
Page load happened
42851 https://kw.com/agent/UPA-6587385291488333824-0
***********************************
https://kw.com/agent/UPA-6618172503444791297-9
Page load happened
42852 https://kw.com/agent/UPA-6618172503444791297-9
***********************************
https://kw.com/agent/UPA-6587385348636540937-4
Page load happened
42853 https://kw.com/agent/UPA-6587385348636540937-4
***********************************
https://kw.com/agent/UPA-6821552292935344128-5
Page load happened
42854 https://kw.com/agent/UPA-6821552292935344128-

Page load happened
42901 https://kw.com/agent/UPA-6587385417524678657-2
***********************************
https://kw.com/agent/UPA-6729038482269024256-3
Page load happened
42902 https://kw.com/agent/UPA-6729038482269024256-3
***********************************
https://kw.com/agent/UPA-6587385002312798209-1
Page load happened
42903 https://kw.com/agent/UPA-6587385002312798209-1
***********************************
https://kw.com/agent/UPA-6587385002061139974-9
Page load happened
42904 https://kw.com/agent/UPA-6587385002061139974-9
***********************************
https://kw.com/agent/UPA-6846163297211686912-2
Page load happened
42905 https://kw.com/agent/UPA-6846163297211686912-2
***********************************
https://kw.com/agent/UPA-6587385103832268802-9
Page load happened
42906 https://kw.com/agent/UPA-6587385103832268802-9
***********************************
https://kw.com/agent/UPA-6587385245584871428-1
Page load happened
42907 https://kw.com/agent/UPA-6587385245584871428-

Page load happened
42954 https://kw.com/agent/UPA-6714557691453636608-8
***********************************
https://kw.com/agent/UPA-6587385076174589960-2
Page load happened
42955 https://kw.com/agent/UPA-6587385076174589960-2
***********************************
https://kw.com/agent/UPA-6788228814120968192-0
Page load happened
42956 https://kw.com/agent/UPA-6788228814120968192-0
***********************************
https://kw.com/agent/UPA-6735621713900236800-3
Page load happened
42957 https://kw.com/agent/UPA-6735621713900236800-3
***********************************
https://kw.com/agent/UPA-6587385369595793412-5
Page load happened
42958 https://kw.com/agent/UPA-6587385369595793412-5
***********************************
https://kw.com/agent/UPA-6826580856204558336-8
Page load happened
42959 https://kw.com/agent/UPA-6826580856204558336-8
***********************************
https://kw.com/agent/UPA-6593007806096314368-6
Page load happened
42960 https://kw.com/agent/UPA-6593007806096314368-